In [59]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pandas as pd

# Load Browser

In [66]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# Define here the URL to scrape

In [67]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

# Get HTML Code from URL

In [68]:
page = requests.get(url)

# Convert to BeautifulSoup object

In [69]:
soup = BeautifulSoup(page.content, 'html.parser')

# Collect latest Title and Text

In [70]:
result = soup.find_all("div", {'class': 'content_title'})
news_title = result[0].text

result = soup.find_all('div', class_='rollover_description_inner')
news_p = result[0].text

print(news_title)
print(news_p)



NASA's Perseverance Team Assessing First Mars Sampling Attempt



The rover continues to explore Jezero Crater while the team assesses today’s activities.



# NASA's JPL Mars Images - Get Featured Image

### NOTE: This part could not be completed  because NASA's JPL site throws "Not Found" error

# Mars Facts

In [74]:
url='https://space-facts.com/mars/'

# get tables using Pandas
tables=pd.read_html(url)
print(tables)

[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers,   Mars - Earth Comparison             Mars            Earth
0               Diameter:         6,779 km        12,742 km
1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
2                  Moons:                2                1
3      Distance from Sun:   227,943,824 km   149,598,262 km
4         Length of Year:   687 Earth days      365.24 days
5            Temperature:     -87 to -5 °C      -88 to 58°C,                       0    

### Show the table in table format

In [81]:
table = tables[0]

# Set column names
table = table.rename(columns={0:'Parameter',1:'Value'})
table

Parameter                          Value
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers

# Mars Hemispheres

In [88]:
main_url ='https://astrogeology.usgs.gov/'
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Visit
browser.visit(url)

# Get HTML
html = browser.html

# Parse HTML
soup = bs(html,'html.parser')

# get hemispheres 
hemispheres = soup.find('div', {'class':'collapsible results'})
hemispheres = hemispheres.find_all('div', {'class':'item'})

hemisphere_image_urls = list() # store urls here

for obj in hemispheres:
    hemisphere = obj.find('div', {'class':'description'})
    title = hemisphere.h3.text
    
    sub_url = hemisphere.a['href']
    
    # Visit image's url
    browser.visit(main_url+sub_url)
    
    # get html
    html = browser.html
    
    # Parse
    soup = bs(html, 'html.parser')
    
    # Get image's url
    image_url = soup.find('li').a['href']
    
    # add to list
    sub_dict = {'title': title, 'img_url': image_url}
    hemisphere_image_urls.append(sub_dict)
    


In [89]:
# Display
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [90]:
browser.quit()